In [1]:
import numpy as np
import pandas as pd
import pickle

import os

## Saving Binary format to text using gensim. Reduction of 300 dimensional did not work
### total_vec argument also specified as 100 or 10 is not reducing the dimension.

### Right now it is converted into text format sucessfuly with 300 dimensions

In [2]:
dirname = os.getcwd()
dirname = os.path.dirname(dirname)
dataset_path = os.path.join(dirname, 'datasets/')
print(dataset_path)

/media/kandy/hdd/master-thesis/constituency-parsing/datasets/


In [3]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format(dataset_path+'GoogleNews-vectors-negative300.bin', binary=True)
model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative300.txt', fvocab=dataset_path+'/google-vocab.txt', binary=False)

In [4]:
# failed attempts
model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative100.txt', binary=False, total_vec=100)
model.save_word2vec_format(dataset_path+'GoogleNews-vectors-negative10.txt', binary=False, total_vec=10)

## Read_csv chunking and using specific columns are also not working
### skip rows worked smoothly
### chunking - memory consumption was less. But later on it raised up and came down (could not understand). When about to store the chunked data , memory got overflowed in the middle
### using specific columns did not reduce memory consumption (reading 11 and 301 columns are the same)

In [2]:
googleVectors =  pd.read_csv(dataset_path+'/GoogleNews-vectors-negative300.txt', sep=' ', header=None, skiprows=[0],
                            usecols=[0,1,2,3,4,5,6,7,8,9,10])

print(googleVectors.head())

print(googleVectors.shape) #(2168995, 11)


     0         1         2         3         4         5         6         7   \
0  </s>  0.001129 -0.000896  0.000319  0.001534  0.001106 -0.001404 -0.000031   
1    in  0.070312  0.086914  0.087891  0.062500  0.069336 -0.108887 -0.081543   
2   for -0.011780 -0.047363  0.044678  0.063477 -0.018188 -0.063965 -0.001312   
3  that -0.015747 -0.028320  0.083496  0.050293 -0.110352  0.031738 -0.014221   
4    is  0.007050 -0.073242  0.171875  0.022583 -0.132812  0.198242  0.112793   

         8         9         10  
0 -0.000420 -0.000576  0.001076  
1 -0.154297  0.020752  0.131836  
2 -0.072266  0.064453  0.086426  
3 -0.089844  0.117676  0.118164  
4 -0.107910  0.071777  0.020874  


In [4]:
toy_google_vectors = googleVectors.iloc[:, 0:11]
print(toy_google_vectors.shape)
toy_google_vectors.to_csv(dataset_path+'GoogleNews-vectors-negative10.txt',header=False, index=False,sep=' ')

(2168995, 11)


In [100]:
reader =  pd.read_csv(dataset_path+'GoogleNews-vectors-negative300.txt', sep=' ', header=None, skiprows=[0],
                            usecols=[0,1,2,3,4,5,6,7,8,9,10], chunksize=50000)

j=0
for chunk in reader:
    print(chunk.shape)
    j = j+1
    chunk.to_csv(dataset_path+'GoogleNews-vectors-negative10_'+str(j)+'.txt',header=False, index=False,sep=' ')

### Use basic File reading and writing. Skipped lines uncessary
#### split the lines and take only first 11 dimensional vector (word and 10 dimensions)
#### Done for 11 dimensions. After that need to do it for 101 dimensions

In [75]:
i = 0
with open(dataset_path+'GoogleNews-vectors-negative300.txt', 'r') as read:
    with open(dataset_path+'GoogleNews-vectors-negative10.txt', 'w') as write:
        contents = read.readlines(5)
        while(len(contents) != 0):
            contents = read.readlines(30000)
            contents = [content.split()[0:11] for content in contents]
            contents = [' '.join(content)+'\n' for content in contents]
            write.writelines(contents)
            #i = i + 1
            #if (i == 10000):
            #    break

In [83]:
google_mini = pd.read_csv(dataset_path+'GoogleNews-vectors-negative10.txt', sep =' ', header=None, prefix='col_')
#print(google_10.head())
print(google_mini.shape)
print(google_mini.columns)

(2168995, 11)


In [89]:
def convert_to_lower(entity):
    value = str(entity)
    return value.lower()
google_mini['col_0'] = google_mini['col_0'].apply(convert_to_lower)

In [95]:
google_mini.drop_duplicates(subset='col_0', keep='last', inplace=True)
print(google_mini.shape)

In [97]:
google_mini.to_csv(dataset_path+'GoogleNews-vectors-negative10.txt', sep=' ', header=False, index=False)

In [98]:
google_mini = pd.read_csv(dataset_path+'GoogleNews-vectors-negative10.txt', sep =' ', header=None, prefix='col_')
#print(google_10.head())
print(google_mini.shape)
print(google_mini.columns)

(1965717, 11)
Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9', 'col_10'],
      dtype='object')


Serialize google corpus and dictionaries to convert word into int and back

In [105]:
googleCorpus = set()
google_mini['col_0'].str.lower().apply(googleCorpus.add)
print(len(googleCorpus))

google_corpus_word_to_int = {}
google_corpus_int_to_word = {}
for word in googleCorpus:
    temp = len(google_corpus_word_to_int)
    google_corpus_word_to_int[word] = temp
    google_corpus_int_to_word[temp] = word
#print(imdb_corpus_word_to_int)
#print(imdb_corpus_int_to_word)

temp = len(google_corpus_word_to_int)


outfile = dataset_path+'google_word_corpus.pic'
with open(outfile, 'wb') as pickle_file:
    pickle.dump([googleCorpus,google_corpus_word_to_int, google_corpus_int_to_word], pickle_file)

1965716
